In [1]:
import sys

import pygame as pg
from pygame.locals import *
import numpy as np

from car import Car
from track import Track

import neat
import math
import time
import pickle

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
maps = [
    {'name': 'track01', 'file': 'tracks/track01.png', 'x_text': 1920 / 2, 'y_text': 1080 / 2, 'distance_text': 50},
    {'name': 'track02', 'file': 'tracks/track02.png', 'x_text': 1716, 'y_text': 400, 'distance_text': 50},
    {'name': 'track03', 'file': 'tracks/track03.png', 'x_text': 300, 'y_text': 810, 'distance_text': 50},
]

In [3]:
class NeatCar(Car):
    MAX_LENGTH = 200
    
    def __init__(self, car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height, top_start_line, bottom_start_line):
        super().__init__(car_sprite, pos_x, pos_y, angle, speed, game_map, border_color, map_width, map_height, top_start_line, bottom_start_line)
        self.radars = []
        self.speed_changes = 0
        self.speed_incremental_average = 1
        self.angles = []
    
    def get_reward(self):
        return self.distance / (Car.CAR_SIZE_X / 2) + self.has_touched_finish() * 10000
    
    
    def check_radar(self, degree):
        length = 0
        x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
        y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
        
        while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH:
            length += 1
            x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
            y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))

        dist = int(pg.math.Vector2(x, y).distance_to(self.center))
        self.radars.append([(x, y), dist])
        
        if degree == 0:
            length = 0
            x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
            y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
            
            while not self.is_collision_points(x, y) and length < NeatCar.MAX_LENGTH + 100:
                length += 1
                x = int(self.center[0] + length * math.cos(math.radians(360 - (self.angle + degree))))
                y = int(self.center[1] + length * math.sin(math.radians(360 - (self.angle + degree))))
            
            dist = int(pg.math.Vector2(x, y).distance_to(self.center))
            self.radars.append([(x, y), dist])
            
        
    def draw(self, screen):
        super().draw(screen)        
        for r in self.radars:
            pg.draw.line(screen, (0, 255, 0), self.center, r[0], 1)
            pg.draw.circle(screen, (0, 255, 0), r[0], 5)

    def update(self):
        super().update()
        
        self.speed_incremental_average = (self.speed_incremental_average * self.speed_changes + self.speed) / (self.speed_changes + 1)
        
        self.radars.clear()
        for d in range(-90, 120, 30):
            self.check_radar(d)

    def get_neat_data(self):
        return [r[1] // 30 for r in self.radars] if len(self.radars) > 0 else [0, 0, 0, 0, 0, 0, 0, 0]
        

In [4]:
def run_simulation(genomes, config, speed = 20, full_screen = True):    
    networks = []
    cars = []

    pg.init()
    
    global current_map_path
    my_track = Track(current_map['file'])
    
    screen = pg.display.set_mode((my_track.map_width, my_track.map_height), (SCALED | RESIZABLE) if not full_screen else FULLSCREEN)
    my_track.load_game_map()
    top_start_line, bottom_start_line = my_track.get_start_line_points()

    clock = pg.time.Clock()

    start_pos_x, start_pos_y, angle = my_track.start_pos

    start_pos_y -= Car.CAR_SIZE_Y // 2

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        networks.append(net)
        g.fitness = 0
        cars.append(
            NeatCar('cars/car2d.png', start_pos_x, start_pos_y, angle, speed, my_track.game_map, my_track.border_color,
                    my_track.width, my_track.height, top_start_line, bottom_start_line))

    font_generation = pg.font.SysFont("Arial", 30)
    font_alive = pg.font.SysFont("Arial", 20)

    global current_generation
    current_generation += 1

    timer = time.time()
    MAX_TIME = 60
    
    while True:
        for event in pg.event.get():
            if event.type == QUIT:
                pg.quit()
                
        no_still_alive = 0

        for i, car in enumerate(cars):
            output = networks[i].activate(car.get_neat_data())
            choice = output.index(max(output))

            car.move(choice)                
                
            if car.is_alive():
                no_still_alive += 1
                car.update()
                genomes[i][1].fitness = car.get_reward()

        if no_still_alive == 0 or time.time() - timer > MAX_TIME:
            break

        screen.blit(my_track.game_map, (0, 0))
        for car in cars:
            if car.is_alive():
                car.draw(screen)

        text_gen = font_generation.render("Generation: " + str(current_generation), True, (0, 0, 0))
        text_gen_rect = text_gen.get_rect()
        text_gen_rect.center = (current_map['x_text'], current_map['y_text'])
        screen.blit(text_gen, text_gen_rect)

        text_alive = font_alive.render("Alive: " + str(no_still_alive), True, (0, 0, 0))
        text_alive_rect = text_alive.get_rect()
        text_alive_rect.center = (current_map['x_text'], current_map['y_text'] + current_map['distance_text'])
        screen.blit(text_alive, text_alive_rect)

        pg.display.flip()
        clock.tick(60)

    # pg.quit()

max_fitness = []

for map in maps:
    current_map = map

    current_generation = 0

    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'alg_neat/neat.config')
    population = neat.Population(config)
    population.add_reporter(neat.StdOutReporter(True))

    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    population.run(run_simulation, 30)
    pg.quit()

    with open(f'alg_neat/best_genome_{map["name"]}.pkl', 'wb') as output:
        pickle.dump(population, output, 1)
        
    best_genome = stats.best_genome()
    max_fitness.append(best_genome.fitness)
    
print(max_fitness)


 ****** Running generation 0 ****** 

Population's average fitness: 5.19111 stdev: 4.95790
Best fitness: 31.03333 - size: (8, 64) - species 1 - id 8
Average adjusted fitness: 0.130
Mean genetic distance 1.073, standard deviation 0.178
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    30     31.0    0.130     0
Total extinctions: 0
Generation time: 3.737 sec

 ****** Running generation 1 ****** 

Population's average fitness: 6.32111 stdev: 6.82715
Best fitness: 31.03333 - size: (8, 64) - species 1 - id 8
Average adjusted fitness: 0.168
Mean genetic distance 1.094, standard deviation 0.173
Population of 30 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    30     31.0    0.168     1
Total extinctions: 0
Generation time: 3.285 sec (3.511 average)

 ****** Running generation 2 ****** 

Population's average fitness: 10.14889 stdev: 1

In [5]:
for i in range(len(maps)):
    current_map = maps[i]
    for j in range(len(maps)):
        if i == j:
            continue
            
        best_genome = None
        with open(f'alg_neat/best_genome_{maps[j]["name"]}.pkl', 'rb') as input:
            best_genome = pickle.load(input).best_genome
            
        run_simulation([(1, best_genome)], config, speed = 20, full_screen = True)

pg.quit()